In [1]:
print('Importing packages...')
from datetime import datetime
import numpy as np    #library for numerical computing support for large arrays
import pandas as pd #library used for working with datasets analysing, cleaning, exploring and manipulating data
import xgboost      #allows to run cross validation at each iteration
import re           #provides regular expression support
import nltk         #provides various text processing libraries with a lot of datasets
from sklearn.metrics import accuracy_score       #an opensource data analysis library
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
nltk.download('stopwords')      #set of commonly used words, used in text mining
nltk.download('punkt')          #learn parameters from a corpus in an unsupervised way. tokenizer divides a text into a list of sentences

Importing packages...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print('Reading data...')
dfTest = pd.read_csv("/content/drive/MyDrive/loan_defaulter/dataset/test_indessa.csv")
dfTrain = pd.read_csv("/content/drive/MyDrive/loan_defaulter/dataset/train_indessa.csv")

dfTrain = dfTrain[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'sub_grade', 'term', 'desc', 'emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc','total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'loan_status']]
dfTest = dfTest[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'sub_grade', 'term', 'desc','emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']]

Reading data...


### Data Transformation

In [4]:

def get_last_week_pay(raw) :
    try :
        return int(re.sub("/content/drive/MyDrive/loan_defaulter", "", raw))
    except :
        return -9999

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
def clean_text(raw_text):
    cleantext = np.nan
    if type(raw_text) == str :
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', raw_text)
        cleantext = cleantext.replace('>', '')
        cleantext = ' '.join(cleantext.split())

        stop_words = set(stopwords.words("english"))
        words = word_tokenize(cleantext)

        filtered_sentence = []

        for w in words:
            if w not in stop_words:
                filtered_sentence.append(w)
        return len(filtered_sentence)

    else :
        return 0


In [5]:

#Data transformation/cleanup
#Strip off textual parts, represent values as numeric values
#it makes sense. Convert the datatype to numeric.

dfTrain['term'] = dfTrain['term'].apply(lambda x : int(re.sub("\D", "", x)))
dfTrain['last_week_pay'] = dfTrain['last_week_pay'].apply(get_last_week_pay)
dfTrain['desc'] = dfTrain['desc'].apply(clean_text)
dfTrain['emp_length'].replace('n/a', '0', inplace=True)
dfTrain['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)
dfTest['term'] = dfTest['term'].apply(lambda x : int(re.sub("\D", "", x)))
dfTest['last_week_pay'] = dfTest['last_week_pay'].apply(get_last_week_pay)
dfTest['desc'] = dfTest['desc'].apply(clean_text)
dfTest['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)

dfTrain['term'] = pd.to_numeric(dfTrain['term'], errors='coerce')
dfTest['term'] = pd.to_numeric(dfTest['term'], errors='coerce')

dfTrain['last_week_pay'] = pd.to_numeric(dfTrain['last_week_pay'], errors='coerce')
dfTest['last_week_pay'] = pd.to_numeric(dfTest['last_week_pay'], errors='coerce')

dfTrain['emp_length'] = pd.to_numeric(dfTrain['emp_length'], errors='coerce')
dfTest['emp_length'] = pd.to_numeric(dfTest['emp_length'], errors='coerce')

dfTrain['sub_grade'] = pd.to_numeric(dfTrain['sub_grade'], errors='coerce')
dfTest['sub_grade'] = pd.to_numeric(dfTest['sub_grade'], errors='coerce')

print('Transform done.')

Transform done.


In [6]:
'''
Feature Engineering
'''

# Separating the member_id column of test dataframe to help create a csv after predictions
test_member_id = pd.DataFrame(dfTest['member_id'])


# Creating target variable pandas series from train dataframe, this will be used by cross validation to calculate
# the accuracy of the model
train_target = pd.DataFrame(dfTrain['loan_status'])


# It's good to create a copy of train and test dataframes. this way we can play around different features as we tune the
# performance of the classifier with important features
selected_cols = ['member_id', 'emp_length', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'sub_grade', 'int_rate', 'annual_inc', 'dti', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'mths_since_last_major_derog', 'last_week_pay', 'tot_cur_bal', 'total_rev_hi_lim', 'tot_coll_amt', 'recoveries', 'collection_recovery_fee', 'term', 'acc_now_delinq', 'collections_12_mths_ex_med']
finalTrain = dfTrain[selected_cols]
finalTest = dfTest[selected_cols]

# How big the loan a person has taken with respect to his earnings, annual income to loan amount ratio
finalTrain['loan_to_income'] = finalTrain['annual_inc']/finalTrain['funded_amnt_inv']
finalTest['loan_to_income'] = finalTest['annual_inc']/finalTest['funded_amnt_inv']


# All these attributes indicate that the repayment was not all hunky-dory. All the amounts caclulated are ratios
# like, recovery to the loan amount. This column gives a magnitude of how much the repayment has gone off course
# in terms of ratios.
finalTrain['bad_state'] = finalTrain['acc_now_delinq'] + (finalTrain['total_rec_late_fee']/finalTrain['funded_amnt_inv']) + (finalTrain['recoveries']/finalTrain['funded_amnt_inv']) + (finalTrain['collection_recovery_fee']/finalTrain['funded_amnt_inv']) + (finalTrain['collections_12_mths_ex_med']/finalTrain['funded_amnt_inv'])
finalTest['bad_state'] = finalTest['acc_now_delinq'] + (finalTest['total_rec_late_fee']/finalTest['funded_amnt_inv']) + (finalTest['recoveries']/finalTest['funded_amnt_inv']) + (finalTest['collection_recovery_fee']/finalTest['funded_amnt_inv']) + (finalTrain['collections_12_mths_ex_med']/finalTest['funded_amnt_inv'])

# For the sake of this model, I have used just a boolean flag if things had gone bad, with this case I didn't see
# a benifit of including above computations
finalTrain.loc[finalTrain['bad_state'] > 0, 'bad_state'] = 1
finalTest.loc[finalTest['bad_state'] > 0, 'bad_state'] = 1


# Total number of available/unused 'credit lines'
finalTrain['avl_lines'] = finalTrain['total_acc'] - finalTrain['open_acc']
finalTest['avl_lines'] = finalTest['total_acc'] - finalTest['open_acc']


# Interest paid so far
finalTrain['int_paid'] = finalTrain['total_rec_int'] + finalTrain['total_rec_late_fee']
finalTest['int_paid'] = finalTest['total_rec_int'] + finalTest['total_rec_late_fee']


# Calculating EMIs paid (in terms of percent)
finalTrain['emi_paid_progress_perc'] = ((finalTrain['last_week_pay']/(finalTrain['term']/12*52+1))*100)
finalTest['emi_paid_progress_perc'] = ((finalTest['last_week_pay']/(finalTest['term']/12*52+1))*100)


# Calculating total repayments received so far, in terms of EMI or recoveries after charge off
finalTrain['total_repayment_progress'] = ((finalTrain['last_week_pay']/(finalTrain['term']/12*52+1))*100) + ((finalTrain['recoveries']/finalTrain['funded_amnt_inv']) * 100)
finalTest['total_repayment_progress'] = ((finalTest['last_week_pay']/(finalTest['term']/12*52+1))*100) + ((finalTest['recoveries']/finalTest['funded_amnt_inv']) * 100)

<ipython-input-6-b730c3bc3b11>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalTrain['loan_to_income'] = finalTrain['annual_inc']/finalTrain['funded_amnt_inv']
<ipython-input-6-b730c3bc3b11>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalTest['loan_to_income'] = finalTest['annual_inc']/finalTest['funded_amnt_inv']
<ipython-input-6-b730c3bc3b11>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [7]:
#Null values in training dataset

null= finalTrain.isnull().sum().sort_values(ascending=False)
total =finalTrain.shape[0]
percent_missing= (finalTrain.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})

print ("Null Values in each column:\n", missing_data.sort_values(by ='Total missing', ascending = False))

Null Values in each column:
                     column name  Total missing  Percent missing
0                     sub_grade         532428         1.000000
1        mths_since_last_record         450305         0.845758
2   mths_since_last_major_derog         399448         0.750239
3        mths_since_last_delinq         272554         0.511908
4                  tot_coll_amt          42004         0.078891
5              total_rev_hi_lim          42004         0.078891
6                   tot_cur_bal          42004         0.078891
7                    emp_length          26891         0.050506
8                    revol_util            287         0.000539
9                     bad_state            228         0.000428
10     total_repayment_progress            113         0.000212
11   collections_12_mths_ex_med             95         0.000178
12                    total_acc             16         0.000030
13               acc_now_delinq             16         0.000030
14         

In [8]:
#Null values in testing dataset

null= finalTest.isnull().sum().sort_values(ascending=False)
total =finalTest.shape[0]
percent_missing= (finalTest.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})

print ("Null Values in each column:\n", missing_data.sort_values(by ='Total missing', ascending = False))

Null Values in each column:
                     column name  Total missing  Percent missing
0                     sub_grade         354951         1.000000
1        mths_since_last_record         300021         0.845246
2   mths_since_last_major_derog         266228         0.750042
3        mths_since_last_delinq         181758         0.512065
4                  tot_coll_amt          28272         0.079650
5              total_rev_hi_lim          28272         0.079650
6                   tot_cur_bal          28272         0.079650
7                    emp_length          17934         0.050525
8                    revol_util            215         0.000606
9                     bad_state            170         0.000479
10     total_repayment_progress             84         0.000237
11   collections_12_mths_ex_med             50         0.000141
12                    total_acc             13         0.000037
13               acc_now_delinq             13         0.000037
14         

In [9]:
def fill_nulls(value):
    cols_fill = ['mths_since_last_record','mths_since_last_major_derog',
                 'mths_since_last_delinq','total_rev_hi_lim','tot_cur_bal',
                 'tot_coll_amt','emp_length','revol_util','collections_12_mths_ex_med',
                 'open_acc','total_acc','acc_now_delinq','avl_lines','loan_to_income',
                 'annual_inc','bad_state','total_repayment_progress']

    if value == -9999:
        for col in cols_fill:
            finalTest.loc[finalTest[col].isnull(), col] = -9999
    else :
        for col in cols_fill:
            finalTest.loc[finalTest[col].isnull(), col] = finalTrain[col].median()

In [10]:
fill_nulls(0)

In [11]:
def fill_nulls(value):
    cols_fill = ['mths_since_last_record','mths_since_last_major_derog',
                 'mths_since_last_delinq','total_rev_hi_lim','tot_cur_bal',
                 'tot_coll_amt','emp_length','revol_util','collections_12_mths_ex_med',
                 'open_acc','total_acc','acc_now_delinq','avl_lines','loan_to_income',
                 'annual_inc','bad_state','total_repayment_progress']
    if value == -9999:
        for col in cols_fill:
            finalTrain.loc[finalTrain[col].isnull(), col] = -9999
    else :
        for col in cols_fill:
            finalTrain.loc[finalTrain[col].isnull(), col] = finalTrain[col].median()

In [12]:
fill_nulls(0)

In [13]:
finalTrain = finalTrain.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_trainScaled = scaler.fit_transform(finalTrain)


In [15]:

# Split train and cross validation sets
X_train, X_test, y_train, y_test = train_test_split(np.array(finalTrain), np.array(train_target), test_size=0.30)
eval_set=[(X_test, y_test)]

In [16]:
print('Initializing xgboost.sklearn.XGBClassifier....')

clf = xgboost.sklearn.XGBClassifier(
    objective="binary:logistic",
    learning_rate=0.09,
    seed=9616,
    max_depth=30,
    gamma=10,
    n_estimators=500)



Initializing xgboost.sklearn.XGBClassifier....


In [17]:
print("Training Started")
clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=eval_set, verbose=True)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred)

Training Started


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.92528
[1]	validation_0-auc:0.93332
[2]	validation_0-auc:0.93759
[3]	validation_0-auc:0.93956
[4]	validation_0-auc:0.94114
[5]	validation_0-auc:0.94222
[6]	validation_0-auc:0.94349
[7]	validation_0-auc:0.94440
[8]	validation_0-auc:0.94494
[9]	validation_0-auc:0.94559
[10]	validation_0-auc:0.94593
[11]	validation_0-auc:0.94643
[12]	validation_0-auc:0.94684
[13]	validation_0-auc:0.94716
[14]	validation_0-auc:0.94753
[15]	validation_0-auc:0.94779
[16]	validation_0-auc:0.94808
[17]	validation_0-auc:0.94845
[18]	validation_0-auc:0.94860
[19]	validation_0-auc:0.94886
[20]	validation_0-auc:0.94902
[21]	validation_0-auc:0.94921
[22]	validation_0-auc:0.94939
[23]	validation_0-auc:0.94950
[24]	validation_0-auc:0.94959
[25]	validation_0-auc:0.94969
[26]	validation_0-auc:0.94985
[27]	validation_0-auc:0.94989
[28]	validation_0-auc:0.94993
[29]	validation_0-auc:0.95007
[30]	validation_0-auc:0.95025
[31]	validation_0-auc:0.95038
[32]	validation_0-auc:0.95054
[33]	validation_0-au

In [18]:
submission_file_name = "Loan_Defaulter_submission"
print("Accuracy: %.10f%%" % (accuracy * 100.0))
submission_file_name = submission_file_name + ("_Accuracy_%.6f" % (accuracy * 100)) + '_'

accuracy_per_roc_auc = roc_auc_score(np.array(y_test).flatten(), y_pred)
print("ROC-AUC: %.10f%%" % (accuracy_per_roc_auc * 100))   #evaluate the performance of a binary classification model
submission_file_name = submission_file_name + ("_ROC-AUC_%.6f" % (accuracy_per_roc_auc * 100))



Accuracy: 91.7253598282%
ROC-AUC: 86.1609816814%


### Final Submission

In [19]:
final_pred = pd.DataFrame(clf.predict_proba(np.array(finalTest)))
dfSub = pd.concat([test_member_id, final_pred.iloc[:, 1:2]], axis=1)
dfSub.rename(columns={1:'loan_status'}, inplace=True)
dfSub.to_csv((("/content/drive/MyDrive/loan_defaulter/%s.csv") % (submission_file_name)), index=False)



In [20]:
# Read the CSV file
df = pd.read_csv(("/content/drive/MyDrive/loan_defaulter/%s.csv")%(submission_file_name))

# Filter the values greater than 0.5 in a column
column_name = 'loan_status'
filtered_df = df[df[column_name] > 0.5]

# Print the filtered values
print(filtered_df[column_name])


# Save the filtered DataFrame to a CSV file
filtered_df.to_csv(r'/content/drive/MyDrive/loan_defaulter/filtered.csv', index=False)

6         0.813377
8         0.810719
9         0.521458
10        0.925026
11        0.617705
            ...   
354928    0.917993
354931    0.902455
354940    0.800527
354948    0.793648
354950    0.584134
Name: loan_status, Length: 72239, dtype: float64
